In [122]:
import uvicorn
from fastapi import FastAPI
import pandas as pd
from typing import List



In [129]:
#Desarrollo API: Propones disponibilizar los datos de la empresa usando el framework FastAPI. Las consultas que propones son las siguientes:
#Deben crear 6 funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

app = FastAPI()

movies_clean = pd.read_csv('movies_dataset_clean.csv')
movies_clean['production_countries'] = movies_clean['production_countries'].apply(eval)
movies_clean['crew'] = movies_clean['crew'].apply(eval)
movies_clean['spoken_languages'] = movies_clean['spoken_languages'].apply(eval)
movies_clean['belongs_to_collection'] = movies_clean['belongs_to_collection'].apply(eval)
movies_clean['revenue'] = movies_clean['revenue'].astype(int)
movies_clean['production_companies'] = movies_clean['production_companies'].apply(eval)
@app.get('/')
async def root():
    return {'message': 'Hello World'}


In [72]:
#def peliculas_duracion( Pelicula: str ): Se ingresa una pelicula. Debe devolver la duracion y el año.
#                   Ejemplo de retorno: X . Duración: x. Año: xx
@app.get('/peliculas_duracion')
def peliculas_duracion(Pelicula: str):
    duracion = movies_clean[movies_clean['title'] == Pelicula]['runtime'].values[0]
    anio = movies_clean[movies_clean['title'] == Pelicula]['release_year'].values[0]
    return f"{Pelicula}. Duración: {duracion} minutos. Año: {anio}"


In [124]:
#def franquicia( Franquicia: str ): Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio
#                    Ejemplo de retorno: La franquicia X posee X peliculas, una ganancia total de x y una ganancia promedio de xx
@app.get('/franquicia/{Franquicia}')
def franquicia(Franquicia: str):
    peliculaNro = 0
    cantidad_peliculas = 0
    ganancia_total = 0
    ganancia_promedio = 0
    for i in movies_clean['belongs_to_collection']:
        peliculaNro += 1
        for franquicia in i:
            if franquicia == Franquicia:
                cantidad_peliculas += 1
                ganancia_total += movies_clean['revenue'][peliculaNro]
    ganancia_promedio = ganancia_total / cantidad_peliculas
    return f"La franquicia {Franquicia} posee {cantidad_peliculas} peliculas, una ganancia total de {ganancia_total} y una ganancia promedio de {ganancia_promedio}"
  


In [45]:
#def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma, en formato lista.

@app.get('/get_director/{nombre_director}')
def get_director(nombre_director):

    peliculas_producidas = 0
    titulo = []
    fecha_lanzamiento = []
    retorno = []
    costo = []
    ganancia = []
    retorno_total = 0

    for i, lista_directores in enumerate(movies_clean['crew']):
        for director in lista_directores:
            if director == nombre_director:
                peliculas_producidas += 1
                retorno_total += movies_clean['return'].values[i]
                titulo.append(movies_clean['title'].values[i])
                fecha_lanzamiento.append(movies_clean['release_date'].values[i])
                retorno.append(movies_clean['return'].values[i])
                costo.append(movies_clean['budget'].values[i])
                ganancia.append(movies_clean['revenue'].values[i])
    lista_peliculas = {'titulo': titulo, 'fecha_lanzamiento': fecha_lanzamiento, 'retorno': retorno, 'costo': costo, 'ganancia': ganancia}
    print(f"El director {nombre_director} produjo {peliculas_producidas} peliculas, con un retorno total de {retorno_total}")
    df_movies = pd.DataFrame(lista_peliculas)

    return df_movies

In [77]:
#def peliculas_idioma( Idioma: str ): Se ingresa un idioma (como están escritos en el dataset, no hay que traducirlos!). Debe devolver la cantidad de películas producidas en ese idioma.
#                   Ejemplo de retorno: X cantidad de películas fueron estrenadas en idioma

@app.get('/peliculas_idioma/{Idioma}')
def peliculas_idioma(Idioma: str):
    
    cantidad_peliculas = 0
    for i in movies_clean['spoken_languages']:
        for idioma in i:
            if idioma == Idioma:
                cantidad_peliculas += 1

    return f"{cantidad_peliculas} cantidad de películas fueron estrenadas en {Idioma}"


In [84]:
#def peliculas_pais( Pais: str ): Se ingresa un país (como están escritos en el dataset, no hay que traducirlos!), retornando la cantidad de peliculas producidas en el mismo.
#                    Ejemplo de retorno: Se produjeron X películas en el país X
@app.get('/peliculas_pais/{Pais}')
def peliculas_pais(Pais: str):
    
    cantidad_peliculas = 0
    for i in movies_clean['production_countries']:
        for pais in i:
            if pais == Pais:
                cantidad_peliculas += 1

    return f"Se produjeron {cantidad_peliculas} películas en el país {Pais}"

In [130]:
#def productoras_exitosas( Productora: str ): Se ingresa la productora, entregandote el revunue total y la cantidad de peliculas que realizo.
#                    Ejemplo de retorno: La productora X ha tenido un revenue de x

@app.get('/productoras_exitosas/{Productora}')
def productoras_exitosas(Productora: str):
    peliculaNro = 0
    cantidad_peliculas = 0
    ganancia_total = 0
    for i in movies_clean['production_companies']:
        peliculaNro += 1
        for productora in i:
            if productora == Productora:
                cantidad_peliculas += 1
                ganancia_total += movies_clean['revenue'][peliculaNro]
    return f"La productora {Productora} ha tenido un revenue de {ganancia_total} y ha realizado {cantidad_peliculas} peliculas"

In [132]:
@app.get('/get_director/{nombre_director}')
def get_director(nombre_director: str):
    peliculas_producidas = 0
    titulo = []
    fecha_lanzamiento = []
    retorno = []
    costo = []
    ganancia = []
    retorno_total = 0

    for i, lista_directores in enumerate(movies_clean['crew']):
        for director in lista_directores:
            if director == nombre_director:
                peliculas_producidas += 1
                retorno_total += movies_clean['return'].values[i]
                titulo.append(movies_clean['title'].values[i])
                fecha_lanzamiento.append(movies_clean['release_date'].values[i])
                retorno.append(movies_clean['return'].values[i])
                costo.append(movies_clean['budget'].values[i])
                ganancia.append(movies_clean['revenue'].values[i])
    lista_peliculas = {'titulo': titulo, 'fecha_lanzamiento': fecha_lanzamiento, 'retorno': retorno, 'costo': costo, 'ganancia': ganancia}
    print(f"El director {nombre_director} produjo {peliculas_producidas} peliculas, con un retorno total de {retorno_total}")
    df_movies = pd.DataFrame(lista_peliculas)

    return df_movies

In [134]:
#pruebo la funcion anterior
get_director('James Cameron')

El director James Cameron produjo 12 peliculas, con un retorno total de nan


,titulo,fecha_lanzamiento,retorno,costo,ganancia
0,True Lies,1994-07-14,3.294630,115000000,378882411
1,Terminator 2: Judgment Day,1991-07-01,5.200000,100000000,520000000
2,The Abyss,1989-08-09,1.285716,70000000,90000098
3,Aliens,1986-07-18,9.908998,18500000,183316455
4,The Terminator,1984-10-26,12.245500,6400000,78371200
5,Titanic,1997-11-18,9.225171,200000000,1845034188
6,Piranha Part Two: The Spawning,1981-01-01,0.000000,145786,0
7,Ghosts of the Abyss,2003-04-10,1.310836,13000000,17040871
8,Aliens of the Deep,2005-01-21,NaN,0,0
9,Avatar,2009-12-10,11.763566,237000000,2787965087
